In [1]:
# from reso_extract import ms_graph
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import sys
from pathlib import Path

# Define the path you want to add
new_path = Path("C:\\Users\\andmn\\PycharmProjects\\kverse")

# Convert Path object to string and append it to sys.path
sys.path.append(str(new_path))
from selenium.webdriver.support.wait import WebDriverWait

from kdags.assets.reparation.reso_old.file_processor import process_docs_data

from kdags.assets.reparation.reso.web_driver import *
from kdags.assets.reparation.reso.main_navigation import *
from kdags.assets.reparation.reso.component_status_scraper import *
import pandas as pd
from selenium.webdriver.common.by import By

from selenium.webdriver.support import expected_conditions as EC
import dagster as dg

context = dg.build_asset_context()

In [2]:
uploaded_uris = []
summary_data = {
    "years_processed": [],
    "uploaded_files": [],
    "failed_years": [],
    "num_files_uploaded": 0,
    "status": "incomplete",
}

num_years_back = 2  # Define how many years back to process
# --- Process Past Years ---
current_date = datetime.now()
current_year = current_date.year
start_loop_year = current_year
end_loop_year = start_loop_year - num_years_back

# --- Initialize Driver ---
context.log.info("Initializing WebDriver...")
driver = initialize_driver()
wait = WebDriverWait(driver, DEFAULT_WAIT)
context.log.info("WebDriver initialized.")

# --- Login ---
context.log.info("Logging into RESO+...")
login_to_reso(driver, wait)
context.log.info("Login successful.")

context.log.info(f"Starting year loop from {start_loop_year} down to {end_loop_year}...")

for year in range(start_loop_year, end_loop_year - 1, -1):
    context.log.info(f"Processing Year: {year}")
    summary_data["years_processed"].append(year)
    start_date_str = f"01-01-{year}"
    end_date_str = f"31-12-{year}"

    # --- Navigate ---
    driver.get(RESO_URL)
    click_reportabilidad(driver, wait)
    click_component_status(driver, wait)
    context.log.info("Navigated to Estatus Componente page.")

    # --- Set Default Filters ---
    set_default_component_status_filters(driver, wait)
    context.log.info("Default filters applied.")

    # Set dates for the current year
    set_component_status_dates(driver, wait, start_date_str, end_date_str)

    # Export the results for the current year
    export_component_status_results(driver, wait)

    # Define upload URI for the current year
    # Using current_date for partition, but year for filename seems intended
    partition_path = current_date.strftime("y=%Y/m=%m/d=%d")  # Partition based on run date
    az_path = f"az://bhp-raw-data/RESO/COMPONENT_STATUS/{partition_path}/component_status_{year}.parquet"

    # Upload results and cleanup
    uploaded_uri = upload_component_status_results(context, az_path, year)
    if uploaded_uri:
        uploaded_uris.append(uploaded_uri)
        # Optionally add more details per file if needed
        summary_data["uploaded_files"].append({"year": year, "az_path": uploaded_uri})
    else:
        context.log.error(f"Upload failed for year {year}")
        summary_data["failed_years"].append(year)
        # Decide if you want to stop or continue if one year fails

context.log.info("Year loop finished.")
summary_data["status"] = "completed" if not summary_data["failed_years"] else "completed_with_errors"
summary_data["num_files_uploaded"] = len(uploaded_uris)

context.log.info("Closing WebDriver...")
driver.quit()
context.log.info("WebDriver closed.")

# Return the summary dictionary
# context.add_output_metadata(metadata=summary_data)

2025-04-20 18:00:36 -0400 - dagster - INFO - system - Initializing WebDriver...
2025-04-20 18:00:38 -0400 - dagster - INFO - system - WebDriver initialized.
2025-04-20 18:00:38 -0400 - dagster - INFO - system - Logging into RESO+...
2025-04-20 18:01:32 -0400 - dagster - INFO - system - Login successful.
2025-04-20 18:01:32 -0400 - dagster - INFO - system - Starting year loop from 2025 down to 2023...
2025-04-20 18:01:32 -0400 - dagster - INFO - system - Processing Year: 2025


--- Clicking Reportabilidad ---
Clicked 'Reportabilidad'.


2025-04-20 18:01:36 -0400 - dagster - INFO - system - Navigated to Estatus Componente page.
2025-04-20 18:01:38 -0400 - dagster - INFO - system - Default filters applied.


--- Setting Dates: 01-01-2025 to 31-12-2025 ---
Setting 'Date From': 01-01-2025
  Target Day: 1
  Clicking calendar icon using JavaScript...
  Sent keys '01-01-2025' to input field.
  Input value confirmed.
  Calendar widget is visible.
  Looking for day '1' link within the calendar widget...
  Found day '1' element. Now waiting for it to be clickable...
  Day '1' is clickable.
  Clicked day '1' in calendar for 'Date From'.
  Calendar widget closed.
Setting 'Date Up': 31-12-2025
  Target Day: 31
  Clicking calendar icon using JavaScript...
  Sent keys '31-12-2025' to input field.
  Input value confirmed.
  Calendar widget is visible.
  Looking for day '31' link within the calendar widget...
  Found day '31' element. Now waiting for it to be clickable...
  Day '31' is clickable.
  Clicked day '31' in calendar for 'Date Up'.
  Calendar widget closed.

Finished setting dates: 01-01-2025 to 31-12-2025
--- Uploading results for year 2025 to az://bhp-raw-data/RESO/COMPONENT_STATUS/y=2025/m=0

2025-04-20 18:01:56 -0400 - dagster - INFO - system - Checking for downloads in: C:\Users\vales\Downloads
2025-04-20 18:01:57 -0400 - dagster - INFO - system - Identified most recent Excel file: C:\Users\vales\Downloads\Components Status (7).xls
2025-04-20 18:01:58 -0400 - dagster - INFO - system - Successfully read data from Components Status (7).xls
2025-04-20 18:02:02 -0400 - dagster - INFO - system - Successfully uploaded data for year 2025 to az://bhp-raw-data/RESO/COMPONENT_STATUS/y=2025/m=04/d=20/component_status_2025.parquet
2025-04-20 18:02:02 -0400 - dagster - INFO - system - Deleting local file: C:\Users\vales\Downloads\Components Status (7).xls
2025-04-20 18:02:02 -0400 - dagster - INFO - system - Deleted local file: Components Status (7).xls
2025-04-20 18:02:02 -0400 - dagster - INFO - system - Processing Year: 2024


...

--- Clicking Reportabilidad ---
Clicked 'Reportabilidad'.


2025-04-20 18:03:54 -0400 - dagster - INFO - system - Navigated to Estatus Componente page.
2025-04-20 18:03:55 -0400 - dagster - INFO - system - Default filters applied.


--- Setting Dates: 01-01-2023 to 31-12-2023 ---
Setting 'Date From': 01-01-2023
  Target Day: 1
  Clicking calendar icon using JavaScript...
  Sent keys '01-01-2023' to input field.
  Input value confirmed.
  Calendar widget is visible.
  Looking for day '1' link within the calendar widget...
  Found day '1' element. Now waiting for it to be clickable...
  Day '1' is clickable.
  Clicked day '1' in calendar for 'Date From'.
  Calendar widget closed.
Setting 'Date Up': 31-12-2023
  Target Day: 31
  Clicking calendar icon using JavaScript...
  Sent keys '31-12-2023' to input field.
  Input value confirmed.
  Calendar widget is visible.
  Looking for day '31' link within the calendar widget...
  Found day '31' element. Now waiting for it to be clickable...
  Day '31' is clickable.
  Clicked day '31' in calendar for 'Date Up'.
  Calendar widget closed.

Finished setting dates: 01-01-2023 to 31-12-2023
--- Uploading results for year 2023 to az://bhp-raw-data/RESO/COMPONENT_STATUS/y=2025/m=0

2025-04-20 18:05:17 -0400 - dagster - INFO - system - Checking for downloads in: C:\Users\vales\Downloads
2025-04-20 18:05:18 -0400 - dagster - INFO - system - Identified most recent Excel file: C:\Users\vales\Downloads\Components Status (7).xls
2025-04-20 18:05:25 -0400 - dagster - INFO - system - Successfully read data from Components Status (7).xls
2025-04-20 18:05:30 -0400 - dagster - INFO - system - Successfully uploaded data for year 2023 to az://bhp-raw-data/RESO/COMPONENT_STATUS/y=2025/m=04/d=20/component_status_2023.parquet
2025-04-20 18:05:30 -0400 - dagster - INFO - system - Deleting local file: C:\Users\vales\Downloads\Components Status (7).xls
2025-04-20 18:05:30 -0400 - dagster - INFO - system - Deleted local file: Components Status (7).xls
2025-04-20 18:05:30 -0400 - dagster - INFO - system - Year loop finished.
2025-04-20 18:05:30 -0400 - dagster - INFO - system - Closing WebDriver...
2025-04-20 18:05:32 -0400 - dagster - INFO - system - WebDriver closed.


DagsterInvalidPropertyError: The add_output_metadata method is not set on the context when an asset or op is directly invoked.